In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_datasets as tfds
import numpy as np

In [10]:
imdb,info=tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train, test=imdb['train'], imdb['test']

training_sentences=[]
training_labels=[]

testing_sentences=[]
testing_labels=[]


for s,l in train:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s,l in test:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

training_labels_final=np.array(training_labels)
testing_labels_final=np.array(testing_labels)


In [19]:
training_sentences[0]

'b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."'

In [11]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


tokenizer=Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index

sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length)

In [31]:
len(training_sentences)

25000

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                9600      
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 169,997
Trainable params: 169,997
Non-trainable params: 0
_________________________________________________________________


In [13]:
num_epochs = 50
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/50
782/782 [==============================] - 65s 83ms/step - loss: 0.5785 - accuracy: 0.6716 - val_loss: 0.3955 - val_accuracy: 0.8265
Epoch 2/50
782/782 [==============================] - 64s 82ms/step - loss: 0.3439 - accuracy: 0.8564 - val_loss: 0.3639 - val_accuracy: 0.8380
Epoch 3/50
782/782 [==============================] - 64s 82ms/step - loss: 0.2675 - accuracy: 0.8967 - val_loss: 0.3644 - val_accuracy: 0.8471
Epoch 4/50
782/782 [==============================] - 65s 83ms/step - loss: 0.2182 - accuracy: 0.9208 - val_loss: 0.4202 - val_accuracy: 0.8266
Epoch 5/50
782/782 [==============================] - 65s 84ms/step - loss: 0.1780 - accuracy: 0.9391 - val_loss: 0.4472 - val_accuracy: 0.8360
Epoch 6/50
782/782 [==============================] - 65s 83ms/step - loss: 0.1469 - accuracy: 0.9516 - val_loss: 0.5349 - val_accuracy: 0.8250
Epoch 7/50
782/782 [==============================] - 65s 84ms/step - loss: 0.1096 - accuracy: 0.9662 - val_loss: 0.5378 - val_accuracy:

In [42]:
#here i picked a random comment for the movie Tenet.

t="The whole movie is confusing. We couldn't understand what half the script was because of the mumbling, accents, fast talking, background noise etc. It's bad. I thought maybe it was just me, but my husband said the same thing afterwards, and now I'm reading on here other reviews talking about that as well."
t=t.split('\n')
seq=tokenizer.texts_to_sequences(t)
pad=pad_sequences(seq, maxlen=max_length, truncating=trunc_type)

pred=model.predict(pad)
np.round(pred)


array([[0.]], dtype=float32)

In [16]:
#We can sse that the predcited sentiment is zero, which means it sia negative review.

(403, 120)

In [17]:
len(t)

403

In [23]:
len(seq)

402